In [1]:
from math import log
import numpy as np

In [2]:
def cal_EAR_from_APR(apr, n):
    """
    Calculate the effective annual rate (EAR) from the annual percentage rate (APR) and the number of compounding periods per year (n).
    """
    return (1 + apr/n) ** n - 1

def cal_APR_from_EAR(ear, n):
    """
    Calculate the annual percentage rate (APR) from the effective annual rate (EAR) and the number of compounding periods per year (n).
    """
    return ((1 + ear) ** (1/n) - 1) * 12


In [3]:
def cal_PV_from_FV(FV, r, n):
    """
    FV: future value
    r: period interest rate
    n: number of periods
    """
    return FV / (1 + r) ** n

# example
FV = 2155000
r = 0.0492 / 12 # 4.92% apr
n = 10 # 10 months
cal_PV_from_FV(FV, r, n)

2068605.1602565811

In [4]:
def cal_period_needed(FV, r, PV):
    """
    FV: future value
    r: period interest rate
    PV: present value
    """
    return log(FV/PV) / log(1+r)

# example
PV = 150725
ear = 0.084
FV = 320000

cal_period_needed(FV, ear, PV)


9.334038987542009

In [5]:
def cal_FV_from_PV(PV, r, n):
    """
    PV: present value
    r: period interest rate
    n: number of periods
    """
    return PV * (1 + r) ** n

# example
PV = 200000
apr = 0.08
n = 10
cal_FV_from_PV(PV, apr, n)

431784.99945455766

In [6]:
def cal_r_from_FV_PV(FV, PV, n):
    """
    FV: future value
    PV: present value
    n: number of periods
    """
    return (FV/PV) ** (1/n) - 1

# example
PV = 10000
n = 3
FV = 12600
cal_r_from_FV_PV(FV, PV, n)

0.08008229825529067

In [7]:
def cal_monthly_payment(PV, r, n):
    """
    PV: present value
    r: period interest rate
    n: number of periods
    """
    return PV * r * (1 + r) ** n / ((1 + r) ** n - 1)

# example
PV = 16000
n = 36
r = 0.054/12
cal_monthly_payment(PV, r, n)

482.413093630601

In [8]:
def cal_FV_with_deposit(term, deposit, r):
    """
    term: number of periods
    deposit: deposit amount each period
    r: period interest rate
    """
    return deposit * ((1 + r) ** term - 1) / r

# example
deposit = 200
terms = 40*12
r = 0.1/12
cal_FV_with_deposit(terms, deposit, r)

1264815.9161838996

Cash Flows to be considered:

Revenue + financing (selling bond or stock) = Expense for Operating + debet service + investment in new assets + Divident

Operating Cash Flow = Revenue - Expense for Operating

In [9]:
def cal_OCF(sale, cost, tax_rate, depreciation):
    """
    sale: sale in the period
    cost: cost in the period
    tax_rate: tax rate
    depreciation: depreciation in the period
    """
    return (sale - cost) * (1 - tax_rate) + depreciation * tax_rate

sale = 2.1 * 1e6
cost = 1.55 * 1e6
tax_rate = 0.3
depreciation = 3 * 1e6 / 6
cal_OCF(sale, cost, tax_rate, depreciation)

535000.0

In [10]:
def cal_IRR(initial_investment, cash_flows, n):
    """
    initial_investment: initial investment
    cash_flows: list of cash flows in each period
    n: number of periods
    """
    irr = 0.1  # Initial guess for IRR
    epsilon = 1e-6  # Convergence tolerance
    max_iterations = 1000  # Maximum number of iterations

    for _ in range(max_iterations):
        npv = -initial_investment
        npv_derivative = 0
        for t in range(1, n + 1):
            npv += cash_flows[t-1] / (1 + irr) ** t
            npv_derivative -= t * cash_flows[t-1] / (1 + irr) ** (t + 1)

        if abs(npv) < epsilon:
            return irr

        irr -= npv / npv_derivative

    raise ValueError("IRR calculation did not converge")

initial_investment = 10 * 1e6
n = 10
cash_flows = [1.25 * 1e6] * n  # Create a list with n same cash flows
cal_IRR(initial_investment, cash_flows, n)

0.04277497803511139

In [11]:
def cal_investment_NPV_IRR(investment, life, salvage, sales, cost, r, tax, depreciation):
    """
    investment: initial investment
    life: life of the project
    salvage: salvage value
    sales: sales revenue
    cost: cost
    r: required rate of return
    tax: tax rate
    depreciation: depreciation amount in each year
    """
    ocf = cal_OCF(sales, cost, tax, depreciation)
    pv_ocf = ocf * (1 - (1 + r) ** -life) / r
    book_value = investment - depreciation * life
    ATSV = salvage - (tax * (salvage - book_value))
    pv_ATSV = ATSV / (1 + r) ** life
    NPV = -investment + pv_ocf + pv_ATSV
    list_ocf = [ocf] * life 
    IRR = cal_IRR(investment, list_ocf, life)
    return NPV, IRR

investment = 12000000
n = 12
depreciation = investment / n
salvage = 1200000
sale = 4150000
cost = 1300000
r = 0.13
tax = 0.28
cal_investment_NPV_IRR(investment, n, salvage, sale, cost, r, tax, depreciation)

(1999282.7305999605, 0.16239606942510681)

Now start the most difficulty part of bonds, the annuity: a stream of constant cash flows (C) with a finite life (t).

While in constrast, perpetuity is a constant cash flow stream with infinite life.

So that basic formula: $PV_{ann} = PV_{reg perp} - PV_{delay perp} = \frac{C}{r}(1-\frac{1}{(1+r)^t})$

In [12]:
def plan_retirement_period_savings(initial_savings, r, n, goal):
    """
    initial_savings: initial savings
    r: effective interest rate for each period
    n: number of periods
    goal: goal amount
    """
    FV_initial = initial_savings * (1 + r) ** n
    FV_needed = goal - FV_initial
    period_savings = FV_needed * r / ((1 + r) ** n - 1)
    return period_savings

initial_savings = 180000
EAR = 0.0925
n = 30 * 12
goal = 4.3 * 1e6

plan_retirement_period_savings(initial_savings, cal_APR_from_EAR(EAR, 12) / 12, n, goal)

975.6180209447112

In [13]:
def cal_r_from_period_payment(period_payment, n, PV, down_payment=0):
    """
    Calculates the periodic effective interest rate using the Bisection Method.

    This function does not use any external packages like numpy. It manually
    implements a numerical solver to find the interest rate.

    Args:
        period_payment (float): The amount of the payment made each period.
        n (int): The total number of payment periods.
        PV (float): The present value or the initial principal of the loan/asset.
        down_payment (float, optional): Any down payment made at the beginning. 
                                        Defaults to 0.

    Returns:
        float: The periodic interest rate, or None if a solution cannot be found.
    """
    loan_amount = PV - down_payment

    # --- Basic Sanity Checks ---
    if n <= 0:
        return None # Number of periods must be positive
    # If total payments are less than the loan, it implies a negative interest rate,
    # which this simple solver isn't designed to handle.
    if period_payment * n <= loan_amount:
        return None

    # --- Objective Function ---
    # We need to find the root `r` for the Present Value of an Annuity formula:
    # f(r) = period_payment * (1 - (1 + r)**-n) / r - loan_amount = 0
    # This is the function whose root (where f(r) = 0) we are trying to find.
    def f(r):
        if r == 0: # Avoid division by zero; the limit of the formula as r->0 is n * pmt
            return (period_payment * n) - loan_amount
        else:
            return period_payment * (1 - (1 + r)**-n) / r - loan_amount

    # --- Bisection Method Implementation ---
    low = 0.0
    high = 1.0  # An upper bound of 100% periodic rate is a safe starting point
    tolerance = 1e-7 # The desired precision for the result
    max_iterations = 100 # A safeguard against an infinite loop

    # The bisection method requires the function to have opposite signs at the
    # low and high ends of the bracket. For a valid loan (r>0), f(low) will be
    # positive and f(high) will be negative.
    if f(low) * f(high) > 0:
        # If signs are the same, a simple root may not exist in this bracket.
        # This can happen for unusual loan terms.
        return None

    for _ in range(max_iterations):
        mid_rate = (low + high) / 2
        f_mid = f(mid_rate)

        # If the value of the function at the midpoint is very close to zero,
        # we have found our root.
        if abs(f_mid) < tolerance:
            return mid_rate

        # If the sign of f(mid_rate) is the same as f(low), it means the root
        # lies in the upper half of the interval. So, we move the lower bound up.
        elif f(low) * f_mid > 0:
            low = mid_rate
        # Otherwise, the root is in the lower half. We move the upper bound down.
        else:
            high = mid_rate
            
    # If the loop completes without converging, return None.
    return None

PV = 425000
n = 360
monthly_payment = 2346.87

cal_r_from_period_payment(monthly_payment, n, PV) * 12

0.052500162056048794

In [14]:
class AnnuityCalculator:
    """
    Comprehensive annuity calculator for various financial applications
    """
    
    @staticmethod
    def present_value_ordinary_annuity(payment, rate, periods):
        """
        Calculate present value of ordinary annuity (payments at end of period)
        PV = C * [1 - (1 + r)^(-t)] / r
        """
        if rate == 0:
            return payment * periods
        return payment * (1 - (1 + rate) ** (-periods)) / rate
    
    @staticmethod
    def present_value_annuity_due(payment, rate, periods):
        """
        Calculate present value of annuity due (payments at beginning of period)
        PV_due = PV_ordinary * (1 + r)
        """
        pv_ordinary = AnnuityCalculator.present_value_ordinary_annuity(payment, rate, periods)
        return pv_ordinary * (1 + rate)
    
    @staticmethod
    def future_value_ordinary_annuity(payment, rate, periods):
        """
        Calculate future value of ordinary annuity
        FV = C * [((1 + r)^t - 1) / r]
        """
        if rate == 0:
            return payment * periods
        return payment * (((1 + rate) ** periods - 1) / rate)
    
    @staticmethod
    def future_value_annuity_due(payment, rate, periods):
        """
        Calculate future value of annuity due
        FV_due = FV_ordinary * (1 + r)
        """
        fv_ordinary = AnnuityCalculator.future_value_ordinary_annuity(payment, rate, periods)
        return fv_ordinary * (1 + rate)
    
    @staticmethod
    def calculate_payment_from_pv(present_value, rate, periods):
        """
        Calculate payment given present value (loan payments)
        C = PV * r / [1 - (1 + r)^(-t)]
        """
        if rate == 0:
            return present_value / periods
        return present_value * rate / (1 - (1 + rate) ** (-periods))
    
    @staticmethod
    def calculate_payment_from_fv(future_value, rate, periods):
        """
        Calculate payment needed to reach future value (savings goal)
        C = FV * r / [(1 + r)^t - 1]
        """
        if rate == 0:
            return future_value / periods
        return future_value * rate / ((1 + rate) ** periods - 1)

class RetirementPlanCalculator:
    """
    Specialized calculator for retirement planning scenarios
    """
    
    def __init__(self, current_age, retirement_age, life_expectancy=85):
        self.current_age = current_age
        self.retirement_age = retirement_age
        self.life_expectancy = life_expectancy
        self.working_years = retirement_age - current_age
        self.retirement_years = life_expectancy - retirement_age
    
    def calculate_retirement_savings_needed(self, annual_income_needed, inflation_rate=0.03, 
                                          real_return_rate=0.04):
        """
        Calculate how much needs to be saved for retirement
        Accounts for inflation during retirement
        """
        # Real return rate adjusted for inflation
        real_rate = (1 + real_return_rate) / (1 + inflation_rate) - 1
        
        # Present value of retirement income needs
        pv_retirement_needs = AnnuityCalculator.present_value_ordinary_annuity(
            annual_income_needed, real_rate, self.retirement_years
        )
        
        return pv_retirement_needs
    
    def calculate_annual_savings_needed(self, retirement_goal, annual_return=0.07):
        """
        Calculate annual savings needed to reach retirement goal
        """
        annual_payment = AnnuityCalculator.calculate_payment_from_fv(
            retirement_goal, annual_return, self.working_years
        )
        
        return annual_payment
    
    def retirement_scenario_analysis(self, annual_income_needed, annual_savings, 
                                   savings_return=0.07, retirement_return=0.04, 
                                   inflation=0.03):
        """
        Complete retirement scenario analysis
        """
        # Future value of savings at retirement
        retirement_fund = AnnuityCalculator.future_value_ordinary_annuity(
            annual_savings, savings_return, self.working_years
        )
        
        # Real return during retirement
        real_retirement_rate = (1 + retirement_return) / (1 + inflation) - 1
        
        # Annual income the fund can provide
        sustainable_income = AnnuityCalculator.calculate_payment_from_pv(
            retirement_fund, real_retirement_rate, self.retirement_years
        )
        
        # Calculate replacement ratio
        replacement_ratio = sustainable_income / annual_income_needed if annual_income_needed > 0 else 0
        
        return {
            'retirement_fund': retirement_fund,
            'sustainable_annual_income': sustainable_income,
            'income_gap': annual_income_needed - sustainable_income,
            'replacement_ratio': replacement_ratio,
            'is_sufficient': sustainable_income >= annual_income_needed
        }

class CreditCardCalculator:
    """
    Calculator for credit card payment scenarios
    """
    
    @staticmethod
    def calculate_minimum_payment_time(balance, annual_rate, min_payment_rate=0.02):
        """
        Calculate time to pay off credit card with minimum payments
        """
        monthly_rate = annual_rate / 12
        min_payment = max(balance * min_payment_rate, 25)  # Minimum $25
        
        if min_payment <= balance * monthly_rate:
            return float('inf')  # Never pays off
        
        months = -log(1 - (balance * monthly_rate) / min_payment) / log(1 + monthly_rate)
        return months
    
    @staticmethod
    def calculate_fixed_payment_schedule(balance, annual_rate, target_months):
        """
        Calculate fixed monthly payment to pay off balance in target months
        """
        monthly_rate = annual_rate / 12
        monthly_payment = AnnuityCalculator.calculate_payment_from_pv(
            balance, monthly_rate, target_months
        )
        
        total_paid = monthly_payment * target_months
        total_interest = total_paid - balance
        
        return {
            'monthly_payment': monthly_payment,
            'total_paid': total_paid,
            'total_interest': total_interest,
            'interest_rate_effective': total_interest / balance
        }
    
    @staticmethod
    def payment_comparison(balance, annual_rate, payment_amounts):
        """
        Compare different payment strategies
        """
        monthly_rate = annual_rate / 12
        results = []
        
        for payment in payment_amounts:
            if payment <= balance * monthly_rate:
                # Payment doesn't cover interest
                months = float('inf')
                total_paid = float('inf')
                total_interest = float('inf')
            else:
                months = -log(1 - (balance * monthly_rate) / payment) / log(1 + monthly_rate)
                total_paid = payment * months
                total_interest = total_paid - balance
            
            results.append({
                'monthly_payment': payment,
                'months_to_payoff': months,
                'total_paid': total_paid,
                'total_interest': total_interest
            })
        
        return results

class MortgageCalculator:
    """
    Mortgage and loan calculator using annuity formulas
    """
    
    @staticmethod
    def calculate_mortgage_payment(loan_amount, annual_rate, loan_term_years):
        """
        Calculate monthly mortgage payment
        """
        monthly_rate = annual_rate / 12
        num_payments = loan_term_years * 12
        
        monthly_payment = AnnuityCalculator.calculate_payment_from_pv(
            loan_amount, monthly_rate, num_payments
        )
        
        return monthly_payment
    
    @staticmethod
    def mortgage_amortization_summary(loan_amount, annual_rate, loan_term_years):
        """
        Calculate mortgage summary statistics
        """
        monthly_payment = MortgageCalculator.calculate_mortgage_payment(
            loan_amount, annual_rate, loan_term_years
        )
        
        num_payments = loan_term_years * 12
        total_paid = monthly_payment * num_payments
        total_interest = total_paid - loan_amount
        
        return {
            'loan_amount': loan_amount,
            'monthly_payment': monthly_payment,
            'total_payments': num_payments,
            'total_paid': total_paid,
            'total_interest': total_interest,
            'interest_percentage': (total_interest / loan_amount) * 100
        }
    
    @staticmethod
    def compare_loan_terms(loan_amount, annual_rate, term_options):
        """
        Compare different loan term options
        """
        comparisons = []
        
        for term in term_options:
            summary = MortgageCalculator.mortgage_amortization_summary(
                loan_amount, annual_rate, term
            )
            summary['term_years'] = term
            comparisons.append(summary)
        
        return comparisons


In [15]:
def retirement_planning_example():
    """
    Example: Retirement planning calculation
    """
    print("=== RETIREMENT PLANNING EXAMPLE ===")
    
    # 30-year-old wanting to retire at 65
    planner = RetirementPlanCalculator(current_age=30, retirement_age=65, life_expectancy=85)
    
    # Wants $80,000 annual income in retirement
    annual_income_needed = 80000
    
    # Calculate retirement fund needed
    retirement_fund_needed = planner.calculate_retirement_savings_needed(
        annual_income_needed, inflation_rate=0.03, real_return_rate=0.04
    )
    
    print(f"Working years: {planner.working_years}")
    print(f"Retirement years: {planner.retirement_years}")
    print(f"Retirement fund needed: ${retirement_fund_needed:,.2f}")
    
    # Calculate annual savings needed
    annual_savings_needed = planner.calculate_annual_savings_needed(
        retirement_fund_needed, annual_return=0.07
    )
    
    print(f"Annual savings needed: ${annual_savings_needed:,.2f}")
    print(f"Monthly savings needed: ${annual_savings_needed/12:,.2f}")
    
    # Scenario analysis with different savings amounts
    savings_scenarios = [15000, 20000, 25000, 30000]
    
    print("\n--- Savings Scenario Analysis ---")
    for savings in savings_scenarios:
        result = planner.retirement_scenario_analysis(
            annual_income_needed, savings
        )
        print(f"Annual savings: ${savings:,}")
        print(f"  Retirement fund: ${result['retirement_fund']:,.2f}")
        print(f"  Sustainable income: ${result['sustainable_annual_income']:,.2f}")
        print(f"  Replacement ratio: {result['replacement_ratio']:.1%}")
        print(f"  Sufficient: {'Yes' if result['is_sufficient'] else 'No'}")
        print()

retirement_planning_example()

=== RETIREMENT PLANNING EXAMPLE ===
Working years: 35
Retirement years: 20
Retirement fund needed: $1,447,886.73
Annual savings needed: $10,473.95
Monthly savings needed: $872.83

--- Savings Scenario Analysis ---
Annual savings: $15,000
  Retirement fund: $2,073,553.18
  Sustainable income: $114,569.91
  Replacement ratio: 143.2%
  Sufficient: Yes

Annual savings: $20,000
  Retirement fund: $2,764,737.57
  Sustainable income: $152,759.88
  Replacement ratio: 190.9%
  Sufficient: Yes

Annual savings: $25,000
  Retirement fund: $3,455,921.96
  Sustainable income: $190,949.85
  Replacement ratio: 238.7%
  Sufficient: Yes

Annual savings: $30,000
  Retirement fund: $4,147,106.35
  Sustainable income: $229,139.82
  Replacement ratio: 286.4%
  Sufficient: Yes



In [16]:
class CarLoanCalculator:
    """
    Specialized calculator for car loans with down payments and early payoff analysis
    """
    
    @staticmethod
    def calculate_car_payment(car_price, down_payment, annual_rate, loan_term_years, 
                             trade_in_value=0, taxes_fees=0):
        """
        Calculate monthly car payment considering all factors
        """
        # Calculate financed amount
        financed_amount = car_price - down_payment - trade_in_value + taxes_fees
        
        if financed_amount <= 0:
            return {
                'financed_amount': financed_amount,
                'monthly_payment': 0,
                'total_payments': 0,
                'total_paid': financed_amount,  # If negative, customer gets money back
                'total_interest': 0
            }
        
        monthly_rate = annual_rate / 12
        num_payments = loan_term_years * 12
        
        monthly_payment = AnnuityCalculator.calculate_payment_from_pv(
            financed_amount, monthly_rate, num_payments
        )
        
        total_paid = monthly_payment * num_payments
        total_interest = total_paid - financed_amount
        
        return {
            'car_price': car_price,
            'down_payment': down_payment,
            'trade_in_value': trade_in_value,
            'taxes_fees': taxes_fees,
            'financed_amount': financed_amount,
            'monthly_payment': monthly_payment,
            'total_payments': num_payments,
            'total_paid': total_paid,
            'total_interest': total_interest,
            'effective_car_cost': car_price + total_interest + taxes_fees - trade_in_value
        }
    
    @staticmethod
    def calculate_remaining_balance(original_loan, annual_rate, total_months, months_paid):
        """
        Calculate remaining balance on a loan after certain payments made
        Using the annuity formula to find present value of remaining payments
        """
        monthly_rate = annual_rate / 12
        monthly_payment = AnnuityCalculator.calculate_payment_from_pv(
            original_loan, monthly_rate, total_months
        )
        
        remaining_months = total_months - months_paid
        
        if remaining_months <= 0:
            return 0
        
        remaining_balance = AnnuityCalculator.present_value_ordinary_annuity(
            monthly_payment, monthly_rate, remaining_months
        )
        
        return remaining_balance
    
    @staticmethod
    def early_payoff_analysis(car_price, down_payment, annual_rate, loan_term_years,
                             early_payoff_month, extra_payment=0, taxes_fees=0):
        """
        Analyze early payoff scenarios
        """
        # Calculate original loan terms
        original_terms = CarLoanCalculator.calculate_car_payment(
            car_price, down_payment, annual_rate, loan_term_years, 0, taxes_fees
        )
        
        if original_terms['financed_amount'] <= 0:
            return original_terms
        
        monthly_rate = annual_rate / 12
        total_months = loan_term_years * 12
        monthly_payment = original_terms['monthly_payment']
        
        # Calculate remaining balance at early payoff month
        remaining_balance = CarLoanCalculator.calculate_remaining_balance(
            original_terms['financed_amount'], annual_rate, total_months, early_payoff_month
        )
        
        # Calculate total paid with early payoff
        regular_payments_made = monthly_payment * early_payoff_month
        total_paid_early = regular_payments_made + remaining_balance + extra_payment
        
        # Calculate savings
        original_total = original_terms['total_paid']
        interest_saved = original_total - total_paid_early
        
        return {
            'original_terms': original_terms,
            'early_payoff_month': early_payoff_month,
            'remaining_balance': remaining_balance,
            'extra_payment': extra_payment,
            'total_early_payoff': remaining_balance + extra_payment,
            'total_paid_with_early_payoff': total_paid_early,
            'interest_saved': interest_saved,
            'months_saved': total_months - early_payoff_month,
            'effective_interest_rate': (total_paid_early - original_terms['financed_amount']) / original_terms['financed_amount']
        }
    
    @staticmethod
    def compare_down_payments(car_price, down_payment_options, annual_rate, 
                             loan_term_years, taxes_fees=0):
        """
        Compare different down payment scenarios
        """
        comparisons = []
        
        for down_payment in down_payment_options:
            # Calculate as percentage if less than 1
            if down_payment < 1:
                actual_down_payment = car_price * down_payment
            else:
                actual_down_payment = down_payment
            
            terms = CarLoanCalculator.calculate_car_payment(
                car_price, actual_down_payment, annual_rate, loan_term_years, 0, taxes_fees
            )
            
            terms['down_payment_percent'] = (actual_down_payment / car_price) * 100
            terms['total_out_of_pocket'] = actual_down_payment + terms['total_paid'] + taxes_fees
            
            comparisons.append(terms)
        
        return comparisons
    
    @staticmethod
    def lease_vs_buy_analysis(car_price, annual_rate, loan_term_years, 
                             lease_monthly_payment, lease_term_months, lease_down=0,
                             residual_value=None, taxes_fees=0, down_payment=0):
        """
        Compare leasing vs buying scenarios
        """
        # Buy scenario
        buy_terms = CarLoanCalculator.calculate_car_payment(
            car_price, down_payment, annual_rate, loan_term_years, 0, taxes_fees
        )
        
        # Lease scenario
        total_lease_payments = lease_monthly_payment * lease_term_months
        total_lease_cost = total_lease_payments + lease_down + taxes_fees
        
        # If residual value provided, calculate cost to purchase at lease end
        if residual_value:
            lease_then_buy_cost = total_lease_cost + residual_value
        else:
            lease_then_buy_cost = None
        
        return {
            'buy_scenario': {
                'down_payment': buy_terms['down_payment'],
                'monthly_payment': buy_terms['monthly_payment'],
                'total_cost': buy_terms['effective_car_cost'],
                'ownership': 'Full ownership after ' + str(loan_term_years) + ' years',
                'car_value_retained': True
            },
            'lease_scenario': {
                'down_payment': lease_down,
                'monthly_payment': lease_monthly_payment,
                'total_cost': total_lease_cost,
                'lease_term_months': lease_term_months,
                'ownership': 'No ownership, return car',
                'lease_then_buy_cost': lease_then_buy_cost
            },
            'cost_difference': buy_terms['effective_car_cost'] - total_lease_cost,
            'monthly_payment_difference': buy_terms['monthly_payment'] - lease_monthly_payment
        }

In [17]:
def car_loan_examples():
    """
    Example: Car loan calculations with different scenarios
    """
    print("=== CAR LOAN CALCULATION EXAMPLES ===")
    
    car_price = 35000
    annual_rate = 0.049  # 4.9% APR
    loan_term = 5  # years
    taxes_fees = 2500
    
    print(f"Car price: ${car_price:,}")
    print(f"Interest rate: {annual_rate:.1%}")
    print(f"Loan term: {loan_term} years")
    print(f"Taxes & fees: ${taxes_fees:,}")
    print()
    
    # Compare different down payments
    down_payment_options = [0, 0.10, 0.15, 0.20, 5000, 10000]  # Mix of percentages and amounts
    
    print("--- Down Payment Comparison ---")
    comparisons = CarLoanCalculator.compare_down_payments(
        car_price, down_payment_options, annual_rate, loan_term, taxes_fees
    )
    
    for comp in comparisons:
        if comp['financed_amount'] <= 0:
            print(f"Down payment: ${comp['down_payment']:,} ({comp['down_payment_percent']:.1f}%)")
            print(f"  No financing needed - paid in full!")
        else:
            print(f"Down payment: ${comp['down_payment']:,} ({comp['down_payment_percent']:.1f}%)")
            print(f"  Financed amount: ${comp['financed_amount']:,}")
            print(f"  Monthly payment: ${comp['monthly_payment']:.2f}")
            print(f"  Total interest: ${comp['total_interest']:,}")
            print(f"  Total out-of-pocket: ${comp['total_out_of_pocket']:,}")
        print()
    
    # Early payoff analysis
    print("--- Early Payoff Analysis ---")
    down_payment = 5000
    
    # Scenarios: pay off after 2, 3, 4 years
    early_payoff_scenarios = [24, 36, 48]  # months
    
    for payoff_month in early_payoff_scenarios:
        analysis = CarLoanCalculator.early_payoff_analysis(
            car_price, down_payment, annual_rate, loan_term, payoff_month, 0, taxes_fees
        )
        
        print(f"Early payoff after {payoff_month} months ({payoff_month/12:.1f} years):")
        print(f"  Remaining balance: ${analysis['remaining_balance']:,.2f}")
        print(f"  Total paid: ${analysis['total_paid_with_early_payoff']:,.2f}")
        print(f"  Interest saved: ${analysis['interest_saved']:,.2f}")
        print(f"  Months saved: {analysis['months_saved']}")
        print()
    
    # Extra payment scenario
    print("--- Extra Payment Strategy ---")
    extra_payment_month = 36
    extra_payment_amount = 2000
    
    analysis_with_extra = CarLoanCalculator.early_payoff_analysis(
        car_price, down_payment, annual_rate, loan_term, 
        extra_payment_month, extra_payment_amount, taxes_fees
    )
    
    print(f"Payoff after {extra_payment_month} months with ${extra_payment_amount:,} extra:")
    print(f"  Remaining balance: ${analysis_with_extra['remaining_balance']:,.2f}")
    print(f"  Extra payment: ${analysis_with_extra['extra_payment']:,}")
    print(f"  Total early payoff: ${analysis_with_extra['total_early_payoff']:,.2f}")
    print(f"  Total interest saved: ${analysis_with_extra['interest_saved']:,.2f}")
    print()
    
    # Lease vs Buy comparison
    print("--- Lease vs Buy Analysis ---")
    lease_monthly = 299
    lease_term = 36
    lease_down = 2000
    residual_value = 18000
    
    lease_buy_analysis = CarLoanCalculator.lease_vs_buy_analysis(
        car_price, annual_rate, loan_term, lease_monthly, lease_term, 
        lease_down, residual_value, taxes_fees, down_payment
    )
    
    print("Buy Option:")
    buy_scenario = lease_buy_analysis['buy_scenario']
    print(f"  Down payment: ${buy_scenario['down_payment']:,}")
    print(f"  Monthly payment: ${buy_scenario['monthly_payment']:.2f}")
    print(f"  Total cost: ${buy_scenario['total_cost']:,.2f}")
    print(f"  Ownership: {buy_scenario['ownership']}")
    print()
    
    print("Lease Option:")
    lease_scenario = lease_buy_analysis['lease_scenario']
    print(f"  Down payment: ${lease_scenario['down_payment']:,}")
    print(f"  Monthly payment: ${lease_scenario['monthly_payment']:.2f}")
    print(f"  Total lease cost: ${lease_scenario['total_cost']:,.2f}")
    print(f"  Lease term: {lease_scenario['lease_term_months']} months")
    print(f"  Lease then buy cost: ${lease_scenario['lease_then_buy_cost']:,.2f}")
    print()
    
    print("Comparison:")
    print(f"  Cost difference (Buy vs Lease): ${lease_buy_analysis['cost_difference']:,.2f}")
    print(f"  Monthly payment difference: ${lease_buy_analysis['monthly_payment_difference']:.2f}")
    
    if lease_buy_analysis['cost_difference'] > 0:
        print("  → Leasing is cheaper short-term")
    else:
        print("  → Buying is more cost-effective")

car_loan_examples()

=== CAR LOAN CALCULATION EXAMPLES ===
Car price: $35,000
Interest rate: 4.9%
Loan term: 5 years
Taxes & fees: $2,500

--- Down Payment Comparison ---
Down payment: $0 (0.0%)
  Financed amount: $37,500
  Monthly payment: $705.95
  Total interest: $4,857.270446155555
  Total out-of-pocket: $44,857.270446155555

Down payment: $3,500.0 (10.0%)
  Financed amount: $34,000.0
  Monthly payment: $640.07
  Total interest: $4,403.925204514366
  Total out-of-pocket: $44,403.925204514366

Down payment: $5,250.0 (15.0%)
  Financed amount: $32,250.0
  Monthly payment: $607.12
  Total interest: $4,177.252583693778
  Total out-of-pocket: $44,177.25258369378

Down payment: $7,000.0 (20.0%)
  Financed amount: $30,500.0
  Monthly payment: $574.18
  Total interest: $3,950.579962873191
  Total out-of-pocket: $43,950.57996287319

Down payment: $5,000 (14.3%)
  Financed amount: $32,500
  Monthly payment: $611.83
  Total interest: $4,209.634386668149
  Total out-of-pocket: $44,209.63438666815

Down payment: $1

In [18]:
def advanced_car_scenarios():
    """
    Advanced car loan scenarios
    """
    print("=== ADVANCED CAR LOAN SCENARIOS ===")
    
    # Scenario 1: Trade-in with negative equity
    print("--- Scenario 1: Trade-in with Upside-down Loan ---")
    new_car_price = 40000
    trade_in_value = 12000
    remaining_loan_balance = 15000  # Owe more than car is worth
    negative_equity = remaining_loan_balance - trade_in_value
    
    # This negative equity gets rolled into new loan
    effective_down_payment = -negative_equity  # Negative down payment
    
    terms = CarLoanCalculator.calculate_car_payment(
        new_car_price, effective_down_payment, 0.059, 6, 0, 1500
    )
    
    print(f"New car price: ${new_car_price:,}")
    print(f"Trade-in value: ${trade_in_value:,}")
    print(f"Remaining loan balance: ${remaining_loan_balance:,}")
    print(f"Negative equity: ${negative_equity:,}")
    print(f"Financed amount: ${terms['financed_amount']:,}")
    print(f"Monthly payment: ${terms['monthly_payment']:.2f}")
    print()
    
    # Scenario 2: GAP insurance calculation
    print("--- Scenario 2: GAP Insurance Value ---")
    car_price = 30000
    down_payment = 3000
    
    # Calculate loan balance after 1 year vs car depreciation
    terms = CarLoanCalculator.calculate_car_payment(car_price, down_payment, 0.048, 5)
    
    remaining_balance_12mo = CarLoanCalculator.calculate_remaining_balance(
        terms['financed_amount'], 0.048, 60, 12
    )
    
    # Assume 20% depreciation in first year
    car_value_12mo = car_price * 0.8
    gap_exposure = remaining_balance_12mo - car_value_12mo
    
    print(f"Original car price: ${car_price:,}")
    print(f"Loan balance after 12 months: ${remaining_balance_12mo:,.2f}")
    print(f"Car value after 12 months: ${car_value_12mo:,.2f}")
    print(f"GAP exposure: ${gap_exposure:,.2f}")
    
    if gap_exposure > 0:
        print(f"  → GAP insurance would cover ${gap_exposure:,.2f}")
    else:
        print("  → No GAP coverage needed")

advanced_car_scenarios()

=== ADVANCED CAR LOAN SCENARIOS ===
--- Scenario 1: Trade-in with Upside-down Loan ---
New car price: $40,000
Trade-in value: $12,000
Remaining loan balance: $15,000
Negative equity: $3,000
Financed amount: $44,500
Monthly payment: $735.39

--- Scenario 2: GAP Insurance Value ---
Original car price: $30,000
Loan balance after 12 months: $22,104.60
Car value after 12 months: $24,000.00
GAP exposure: $-1,895.40
  → No GAP coverage needed


In [19]:
def credit_card_payoff_example():
    """
    Example: Credit card payoff strategies
    """
    print("=== CREDIT CARD PAYOFF EXAMPLE ===")
    
    balance = 8000
    annual_rate = 0.1899  # 18.99% APR
    
    # Calculate minimum payment scenario
    min_months = CreditCardCalculator.calculate_minimum_payment_time(balance, annual_rate)
    
    print(f"Credit card balance: ${balance:,}")
    print(f"Annual interest rate: {annual_rate:.2%}")
    print(f"Time with minimum payments: {min_months/12:.1f} years")
    
    # Compare different payment amounts
    payment_amounts = [160, 200, 300, 400, 500]  # Monthly payments
    
    print("\n--- Payment Strategy Comparison ---")
    comparisons = CreditCardCalculator.payment_comparison(balance, annual_rate, payment_amounts)
    
    for comp in comparisons:
        if comp['months_to_payoff'] == float('inf'):
            print(f"${comp['monthly_payment']:,}/month: Never pays off (payment too low)")
        else:
            print(f"${comp['monthly_payment']:,}/month: {comp['months_to_payoff']:.1f} months, "
                  f"Total paid: ${comp['total_paid']:,.2f}, "
                  f"Interest: ${comp['total_interest']:,.2f}")
    
    # Calculate payment for specific payoff timeline
    target_months = 24  # 2 years
    payoff_plan = CreditCardCalculator.calculate_fixed_payment_schedule(
        balance, annual_rate, target_months
    )
    
    print(f"\n--- Pay off in {target_months} months ---")
    print(f"Required monthly payment: ${payoff_plan['monthly_payment']:.2f}")
    print(f"Total paid: ${payoff_plan['total_paid']:.2f}")
    print(f"Total interest: ${payoff_plan['total_interest']:.2f}")

credit_card_payoff_example()

=== CREDIT CARD PAYOFF EXAMPLE ===
Credit card balance: $8,000
Annual interest rate: 18.99%
Time with minimum payments: 8.3 years

--- Payment Strategy Comparison ---
$160/month: 99.8 months, Total paid: $15,964.42, Interest: $7,964.42
$200/month: 63.8 months, Total paid: $12,768.46, Interest: $4,768.46
$300/month: 34.9 months, Total paid: $10,474.08, Interest: $2,474.08
$400/month: 24.2 months, Total paid: $9,694.32, Interest: $1,694.32
$500/month: 18.6 months, Total paid: $9,297.38, Interest: $1,297.38

--- Pay off in 24 months ---
Required monthly payment: $403.23
Total paid: $9677.52
Total interest: $1677.52


In [20]:
def mortgage_comparison_example():
    """
    Example: Mortgage comparison
    """
    print("=== MORTGAGE COMPARISON EXAMPLE ===")
    
    loan_amount = 400000
    annual_rate = 0.065  # 6.5%
    
    # Compare different loan terms
    terms = [15, 20, 25, 30]
    
    print(f"Loan amount: ${loan_amount:,}")
    print(f"Interest rate: {annual_rate:.1%}")
    print()
    
    comparisons = MortgageCalculator.compare_loan_terms(loan_amount, annual_rate, terms)
    
    print("--- Loan Term Comparison ---")
    for comp in comparisons:
        print(f"{comp['term_years']}-year loan:")
        print(f"  Monthly payment: ${comp['monthly_payment']:,.2f}")
        print(f"  Total paid: ${comp['total_paid']:,.2f}")
        print(f"  Total interest: ${comp['total_interest']:,.2f}")
        print(f"  Interest as % of loan: {comp['interest_percentage']:.1f}%")
        print()

mortgage_comparison_example()


=== MORTGAGE COMPARISON EXAMPLE ===
Loan amount: $400,000
Interest rate: 6.5%

--- Loan Term Comparison ---
15-year loan:
  Monthly payment: $3,484.43
  Total paid: $627,197.30
  Total interest: $227,197.30
  Interest as % of loan: 56.8%

20-year loan:
  Monthly payment: $2,982.29
  Total paid: $715,750.21
  Total interest: $315,750.21
  Interest as % of loan: 78.9%

25-year loan:
  Monthly payment: $2,700.83
  Total paid: $810,248.59
  Total interest: $410,248.59
  Interest as % of loan: 102.6%

30-year loan:
  Monthly payment: $2,528.27
  Total paid: $910,177.95
  Total interest: $510,177.95
  Interest as % of loan: 127.5%



In [21]:
def annuity_formula_examples():
    """
    Direct examples of annuity formulas
    """
    print("=== ANNUITY FORMULA EXAMPLES ===")
    
    # Example 1: Present value of annuity
    payment = 1000
    rate = 0.05
    periods = 10
    
    pv_ordinary = AnnuityCalculator.present_value_ordinary_annuity(payment, rate, periods)
    pv_due = AnnuityCalculator.present_value_annuity_due(payment, rate, periods)
    
    print(f"Payment: ${payment:,}, Rate: {rate:.1%}, Periods: {periods}")
    print(f"PV of ordinary annuity: ${pv_ordinary:,.2f}")
    print(f"PV of annuity due: ${pv_due:,.2f}")
    print(f"Difference: ${pv_due - pv_ordinary:,.2f}")
    print()
    
    # Example 2: Future value of annuity
    fv_ordinary = AnnuityCalculator.future_value_ordinary_annuity(payment, rate, periods)
    fv_due = AnnuityCalculator.future_value_annuity_due(payment, rate, periods)
    
    print(f"FV of ordinary annuity: ${fv_ordinary:,.2f}")
    print(f"FV of annuity due: ${fv_due:,.2f}")
    print(f"Difference: ${fv_due - fv_ordinary:,.2f}")
    print()
    
    # Example 3: Calculate payment needed
    target_fv = 100000
    payment_needed = AnnuityCalculator.calculate_payment_from_fv(target_fv, rate, periods)
    
    print(f"To accumulate ${target_fv:,} in {periods} years at {rate:.1%}:")
    print(f"Annual payment needed: ${payment_needed:,.2f}")

annuity_formula_examples()

=== ANNUITY FORMULA EXAMPLES ===
Payment: $1,000, Rate: 5.0%, Periods: 10
PV of ordinary annuity: $7,721.73
PV of annuity due: $8,107.82
Difference: $386.09

FV of ordinary annuity: $12,577.89
FV of annuity due: $13,206.79
Difference: $628.89

To accumulate $100,000 in 10 years at 5.0%:
Annual payment needed: $7,950.46
